Import and setup some auxiliary functions

In [4]:
import torch
from torchvision import datasets, transforms
import numpy as np
import timeit
from collections import OrderedDict
from pprint import pformat


def compute_score(acc, min_thres, max_thres):
    if acc <= min_thres:
        base_score = 0.0
    elif acc >= max_thres:
        base_score = 100.0
    else:
        base_score = float(acc - min_thres) / (max_thres - min_thres) \
                     * 100
    return base_score


def run(algorithm, x_train, y_train, x_test, y_test, n_classes, device):
    print('Running...')        
    if device != 'cpu' and torch.cuda.is_available():
        device = torch.device("cuda")
        print('Training on GPU: {}'.format(torch.cuda.get_device_name(0)))
    else:
        device = torch.device("cpu")
        print('Training on CPU')
        
    start = timeit.default_timer()
    np.random.seed(0)
    predicted_y_test = algorithm(x_train, y_train, x_test, n_classes, device)
    np.random.seed()
    stop = timeit.default_timer()
    run_time = stop - start

    correct_predict = (y_test
                       == predicted_y_test).astype(np.int32).sum()
    incorrect_predict = len(y_test) - correct_predict
    accuracy = float(correct_predict) / len(y_test)

    print('Correct Predict: {}/{} total \tAccuracy: {:5f} \tTime: {:2f}'.format(correct_predict,
                                                                                len(y_test), accuracy, run_time))
    return correct_predict, accuracy, run_time

TODO: Implement knn here

In [5]:
def knn(x_train, y_train, x_test, n_classes, device):
    """
    x_train: 60000 x 784 matrix: each row is a flattened image of an MNIST digit
    y_train: 60000 vector: label for x_train
    x_test: 1000 x 784 testing images
    n_classes: no. of classes in the classification task
    device: pytorch device on which to run the code
    return: predicted y_test which is a 1000-sized vector
    """
    k = 6
    
    y_pred = torch.zeros(len(x_test))

    x_traingpu = torch.tensor(x_train/255, dtype=torch.float, device=device)
    x_testgpu = torch.tensor(x_test/255, dtype=torch.float, device=device)
    y_traingpu = torch.tensor(y_train, dtype=torch.float, device=device)

    for i in range(len(x_test)):
        # tried three distance functions

        # calculating Euclidean distance
        diffr2=torch.square(x_traingpu - x_testgpu[i])
        dist = torch.sqrt(torch.sum(diffr2,1))
        #dist = np.sum(np.abs(x_traingpu-x_testgpu[i]), axis=1)
        #dist =np.sum(x_traingpu*x_testgpu[i],axis=1)/(np.sqrt(np.sum(np.square(x_traingpu), axis = 1))*np.sqrt(np.sum(np.square(x_testgpu[i]))))
        
        # find k closest indices
        kidx = (torch.topk(dist,k,largest=False))[1]

        # compare closest items to training data, find out the 
        # most appeared label, put it in to the y_pred list
        labels = torch.gather(y_traingpu,0,kidx)
        one_mtrix = torch.nn.functional.one_hot(labels.to(torch.int64),10)
        counts = torch.sum(one_mtrix,0)
        y_pred[i] = torch.argmax(counts).float()
    
    # return predicted value, using .cpu().numpy() to avoid cuda:0 TypeError
    return y_pred.cpu().numpy()
    raise NotImplementedError

Main loop. You can only run this after filling the knn function above

In [6]:
min_thres = 0.84
max_thres = 0.94
n_classes = 10
# change to cpu to run on CPU
device = 'gpu'

mnist_train = datasets.MNIST('data', train=True, download=True,
                             transform=transforms.Compose([
                                 transforms.Normalize((0.1307,), (0.3081,)),
                             ])
                             )
mnist_test = datasets.MNIST('data', train=False, download=True,
                             transform=transforms.Compose([
                                 transforms.Normalize((0.1307,), (0.3081,)),
                             ])
                            )
result = [OrderedDict(first_name='Insert your First name here',
                      last_name='Insert your Last name here')]

# convert pytorch tensors to numpy arrays
(x_train, y_train) = (mnist_train.data.cpu().numpy(), mnist_train.targets.cpu().numpy())
(x_valid, y_valid) = (mnist_test.data.cpu().numpy(), mnist_test.targets.cpu().numpy())

# flatten 28x28 images into 784 sized vectors
x_train = x_train.reshape(x_train.shape[0], -1)
x_valid = x_valid.reshape(x_valid.shape[0], -1)

# You may want to use a smaller training set to save time when debugging
# i.e.: Put something like:
# (x_train, y_train) = (x_train[:5000], y_train[:5000])

# For this assignment, we only test on the first 1000 samples of the test set
(x_valid, y_valid) = (x_valid[:1000], y_valid[:1000])

print("Dimension of dataset: ")
print("Train:", x_train.shape, y_train.shape, "\nTest:", x_valid.shape, y_valid.shape)

(correct_predict, accuracy, run_time) = run(knn, x_train, y_train, x_valid, y_valid, n_classes, device)
score = compute_score(accuracy, min_thres, max_thres)
result = OrderedDict(correct_predict=correct_predict,
                     accuracy=accuracy, score=score,
                     run_time=run_time)

with open('result.txt', 'w') as f:
    f.writelines(pformat(result, indent=4))

print(pformat(result, indent=4))

Dimension of dataset: 
Train: (60000, 784) (60000,) 
Test: (1000, 784) (1000,)
Running...
Training on GPU: Tesla T4
Correct Predict: 962/1000 total 	Accuracy: 0.962000 	Time: 4.717577
OrderedDict([   ('correct_predict', 962),
                ('accuracy', 0.962),
                ('score', 100.0),
                ('run_time', 4.717577235000007)])
